# Setup

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
original_image = cv2.imread('test.jpg')

image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
plt.imshow(image_rgb)
plt.axis('off')
plt.show()